In [1]:
# imports
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd

from robot_functions import *
from crate_rover_specs import *

rover_specs = RoverSpecification()
rover = Rover(rover_specs)

## Drive system

In [2]:
# What are the deflections allowed in the drive system
# 
# What are the results of deflection
# - Potential odometer inaccuracy -> depending on where we measure
# - Position accuracy -> Linked to odometer accuracy
# 
# Accuracy requirements: no more than 50mm from target
position_accuracy = 0.050

rotational_accuracy_wheel = position_accuracy / ( 2 * rover_specs.chassis.wheel.radius_in_meters * math.pi) * 360.0
rotational_accuracy_wheel

38.197186342054884

In [6]:
material = materials[0] # Stainless steel

# Aiming for a total reduction of 16 with 2 gear sets
gear_ratio_per_gear = 4.0
number_of_gears = 2

# The required torque at the wheel
torque_at_wheel = 9.9

# There are 3 shafts in the current design
#
# 1) Motor to bevel gear
# 2) Bevel gear to differential
# 3) Differential to wheel
#
# Each of these handle different amounts of torque
maximum_allowed_shaft_deflection = 5.0

# 1) Motor to bevel gear - Shaft dimensions
length = 0.10
torque_at_motor = torque_at_wheel / 12
diameter = find_solid_shaft_diameter_for_torque(torque_at_motor, length, maximum_allowed_shaft_deflection, material.yield_strength, material.shear_modulus)
deflection = torsional_deflection_in_degrees_of_shaft(diameter / 2.0, 0.0, length, material.shear_modulus, torque_at_motor)
print(f'Motor to bevel gear - shaft: {diameter}. Maximum deflection: {deflection}')

# 2) Bevel gear to pulley
length = 0.10
torque_at_bevel_gear = torque_at_wheel / 4.0
diameter = find_solid_shaft_diameter_for_torque(torque_at_bevel_gear, length, maximum_allowed_shaft_deflection, material.yield_strength, material.shear_modulus)
diameter_middle = Engineering.safety_factor_structural() * diameter
deflection_middle = torsional_deflection_in_degrees_of_shaft(diameter_middle / 2.0, 0.0, length, material.shear_modulus, torque_at_bevel_gear)
print(f'Bevel gear to differential - shaft: {diameter_middle}. Maximum deflection: {deflection_middle}')

# 3) Pulley to wheel
length = 0.10
diameter = find_solid_shaft_diameter_for_torque(torque_at_wheel, length, maximum_allowed_shaft_deflection, material.yield_strength, material.shear_modulus)
diameter_wheel_end = Engineering.safety_factor_structural() * diameter
deflection_wheel_end = torsional_deflection_in_degrees_of_shaft(diameter_wheel_end / 2.0, 0.0, length, material.shear_modulus, torque_at_wheel)
print(f'Differential to wheel - shaft: {diameter_wheel_end}. Maximum deflection: {deflection_wheel_end}')




Motor to bevel gear - shaft: 0.006. Maximum deflection: 0.4586555638130518
Bevel gear to differential - shaft: 0.0075. Maximum deflection: 0.5635959568134781
Differential to wheel - shaft: 0.01. Maximum deflection: 0.713301132842058


## Steering system

In [4]:
# What are the deflections allowed in the steering system
# 
# What are the results of deflection
# - Potential directional inaccuracy, depending on where we measure the position
# - Both directions

In [5]:
torque_at_motor = 1.0
torque_at_wheel = 19.3 * 1.25